# 1. Basic Settings

In [1]:
import gensim
import os
import logging

# Log settings
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


c:\nlp\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
# Note that this imports ALL FILES in dir
class MySentences(object):

    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield gensim.utils.simple_preprocess(line)

# a memory - friendly iterator
sentences = MySentences('D:\\coca and GloWbE\\coca\\temp\\text2doc')



# 2. (skip) Preprocessing - Dividing text in file into Docs

Each file has hundreds of documents, with number tags leading the start of each article 
i.e) ##300000, ##300001, #300002 . . .

In [6]:
for i in range(1990, 2013): # reads each text at directory and prints a few lines
    file = open("D:\\COCA & GloWbE\\COCA\\texts\\text_magazine_qso\\w_mag_%d.txt" % i)
    newtext = file.read()
    print("File", str(i), ":", newtext[1:100], "\n")
    file.close()

File 1990 : ##2018805 Section : WALL STREET What the pros say <p> Does a double-digit yield on a safe stock sou 

File 1991 : ##2018873 Section : MONEY SCORECARD Where to save , borrow , invest <p> Neither President Bush 's r 

File 1992 : ##2000164 The greatest conquest begins to wither the moment it is achieved . So as a quiet reminder 

File 1993 : ##2000178 She kicked off her shoes to dance . Felicia Hendricks liked to dance and this was a party 

File 1994 : ##2000195 Suppose you had spent the best years of your life gasping for oxygen . Suppose you had aw 

File 1995 : ##2000222 Andre Agassi hurtles through the night and across the Nevada desert looking for heat ligh 

File 1996 : ##2000250 // We begin way over there , out on the margin . We begin with a dirty , disheveled 18-ye 

File 1997 : ##2000264 // HE SUDDEN and furious sequence of punches grew out of nowhere , without warning , in t 

File 1998 : ##2000292 // No one has ever chased his baseball dreams with more purity of 

# Segement each yearly file to seperate documents

In [21]:
import re

doc_index = 1

for i in range(1990, 2013):
    file = open("D:\\COCA & GloWbE\\COCA\\texts\\text_magazine_qso\\w_mag_%d.txt" % i)
    newtext = file.read()
    
    # close file after reading
    file.close()
    
    text_ids = re.findall('\n##\d+', newtext)
    
    # Split file between " (line break) hash hash number+ "
    p = re.split('\n##\d+', newtext)
    p = p[1:] # due to list.split() the 1st item is blank
    p = list(zip(text_ids, p)) # zip text ids & text
    p = [''.join(item) for item in p ]
    
    print("File w_mag_%d" % i, "-", "Number of files : ", len(p))

    
    for f in p: # Writes out each element of list (=article) as a new file
        
        new_file = open(("D:\\COCA & GloWbe\\COCA\\Pre-processed text\\Doc2Vec Docs - Mag\\mag_%d.txt" % doc_index), 'w')
        new_file.write(f)

        doc_index += 1
        
print("Total number of files:", doc_index)

File w_mag_1990 - Number of files :  1696
File w_mag_1991 - Number of files :  1735
File w_mag_1992 - Number of files :  1836
File w_mag_1993 - Number of files :  1832
File w_mag_1994 - Number of files :  1817
File w_mag_1995 - Number of files :  1949
File w_mag_1996 - Number of files :  2002
File w_mag_1997 - Number of files :  2230
File w_mag_1998 - Number of files :  2245
File w_mag_1999 - Number of files :  2249
File w_mag_2000 - Number of files :  2248
File w_mag_2001 - Number of files :  2262
File w_mag_2002 - Number of files :  2345
File w_mag_2003 - Number of files :  2453
File w_mag_2004 - Number of files :  2406
File w_mag_2005 - Number of files :  2468
File w_mag_2006 - Number of files :  2351
File w_mag_2007 - Number of files :  2274
File w_mag_2008 - Number of files :  2616
File w_mag_2009 - Number of files :  2956
File w_mag_2010 - Number of files :  2943
File w_mag_2011 - Number of files :  4045
File w_mag_2012 - Number of files :  2240
Total number of files: 53199


# 3. Extracting Science-related article numbers

## 3.1 COCA excel sheet
- containins year, domain, date, author, etc. for each text  

In [28]:
import xlrd

In [29]:
workbook = xlrd.open_workbook('D:\COCA & GloWbE\COCA\coca_2015_12_23.xlsx')

In [38]:
# Print out sheet name
for i, sheet in enumerate(workbook.sheets()):
    print("Sheet",i+1 ,":" , sheet.name)

Sheet 1 : introduction
Sheet 2 : TOTALS
Sheet 3 : spoken
Sheet 4 : fiction
Sheet 5 : magazine
Sheet 6 : newspaper
Sheet 7 : academic
Sheet 8 : Sheet1


### Magazine excel sheet 

In [68]:
mag_sheet = workbook.sheets()[4]

# Headers - column "domain" is at 4th col 
print(mag_sheet.row_values(0))

['textID', '# words', 'Year', 'Domain', 'Magazine', 'Date', 'Publication Info', 'Tile', '', '']


Our column of interest 'domain' is the 4th column. <br>
We iterate through each row and check the 4th column for science related topics
- Science/Tech 
- PopScience 
- ScienceNews


In [85]:
sci_text_rows = []

for i in range(mag_sheet.nrows):
    
    row = mag_sheet.row_values(i)
    
    if(("Science" in row[3]) & ("SocialScience" not in row[3])): # row[3] == 'domain'
        sci_text_rows.append(i)
#    print(mag_sheet.col_values(i)) # mag_sheet.col_values()

Row numbers of entries with science related topics

In [86]:
sci_text_rows

[1393,
 1394,
 1395,
 1396,
 1397,
 1398,
 1399,
 1400,
 1401,
 1402,
 1403,
 1404,
 1405,
 1406,
 1407,
 1408,
 1409,
 1410,
 1411,
 1412,
 1413,
 1414,
 1415,
 1416,
 1417,
 1418,
 1419,
 1420,
 1421,
 1422,
 1423,
 1424,
 1425,
 1426,
 1427,
 1428,
 1429,
 1430,
 1431,
 1432,
 1433,
 1434,
 1435,
 1436,
 1437,
 1438,
 1439,
 1440,
 1441,
 1442,
 1443,
 1444,
 1445,
 1446,
 1447,
 1448,
 1449,
 1450,
 1451,
 1452,
 1453,
 1454,
 1455,
 1456,
 1457,
 1458,
 1459,
 1460,
 1461,
 1462,
 1463,
 1464,
 1465,
 1466,
 1467,
 1468,
 1469,
 1470,
 1471,
 1472,
 1473,
 1474,
 1475,
 1476,
 1477,
 1478,
 1479,
 1480,
 1481,
 1482,
 1483,
 1484,
 1485,
 1486,
 1487,
 2964,
 2965,
 2966,
 2967,
 2968,
 2969,
 2970,
 2971,
 2972,
 2973,
 2974,
 2975,
 2976,
 2977,
 2978,
 2979,
 2980,
 2981,
 2982,
 2983,
 2984,
 2985,
 2986,
 2987,
 2988,
 2989,
 2990,
 2991,
 2992,
 2993,
 2994,
 2995,
 2996,
 2997,
 2998,
 2999,
 3000,
 3001,
 3002,
 3003,
 3004,
 3005,
 3006,
 3007,
 3008,
 3009,
 3010,
 3011,

In [87]:
len(sci_text_rows)

7487

Put the relevant rows in a list

In [116]:
scienceMagTexts_list = [mag_sheet.row_values(i) for i in sci_text_rows]

In [117]:
scienceMagTexts_list

[[2024318.0,
  6385.0,
  1990.0,
  'Science/Tech',
  'Omni',
  'Aug',
  'Crime bytes back. (cover story)',
  'Marsa, L.<BR>Ray, D.',
  'Vol. 12 Issue 11, p34, 8p, 2 illustrations',
  'COMPUTER crimes'],
 [2024319.0,
  2447.0,
  1990.0,
  'Science/Tech',
  'Omni',
  'Aug',
  'Trouble in paradise.',
  'Gorman, P.<BR>Rotman, J.',
  'Vol. 12 Issue 11, p40, 5p, 7c',
  'INDIANS of South America'],
 [2024320.0,
  3611.0,
  1990.0,
  'Science/Tech',
  'Omni',
  'Aug',
  'The catch.',
  '',
  'Vol. 12 Issue 11, p44, 6p, 2 illustrations',
  'SHORT story'],
 [2024322.0,
  1166.0,
  1990.0,
  'Science/Tech',
  'Omni',
  'Aug',
  'Once upon a time in America.',
  'Berger, B.',
  'Vol. 12 Issue 11, p50, 8p, 16c',
  'INVENTIONS'],
 [2024278.0,
  2841.0,
  1990.0,
  'Science/Tech',
  'Omni',
  'Dec',
  'The big ear. (cover story)',
  'Overbye, D.',
  'Vol. 13 Issue 3, p42, 5p, 3c',
  ''],
 [2024279.0,
  5432.0,
  1990.0,
  'Science/Tech',
  'Omni',
  'Dec',
  'Snapshots from the butterfly plague.',
  

Check if any rows have different lengths with each other

In [149]:
temp = [str(i) for i in scienceMagTexts_list[0]]
temp = '\n'.join(temp)
print("A normal row would have 10 elements, as: \n\n", temp)

A normal row would have 10 elements, as: 
 2024318.0
6385.0
1990.0
Science/Tech
Omni
Aug
Crime bytes back. (cover story)
Marsa, L.<BR>Ray, D.
Vol. 12 Issue 11, p34, 8p, 2 illustrations
COMPUTER crimes


In [118]:
for i in range(len(scienceMagTexts_list)):
    
    if i == 0: 
        temp = len(scienceMagTexts_list[i])
        pass
    
    print(temp == len(scienceMagTexts_list[i]))
    
    temp = len(scienceMagTexts_list[i])
    

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


### Now to write out a new excel file with only the relevant entries

In [172]:
import xlsxwriter
processed_workbook = xlsxwriter.Workbook(r'D:\COCA & GloWbE\COCA\COCA Mag Science texts')
worksheet = processed_workbook.add_worksheet()

In [173]:
original_sheet_COLS = len(mag_sheet.row_values(0))
target_sheet_ROWS = len(scienceMagTexts_list)
target_sheet_COLS = len(scienceMagTexts_list[0])

# Write header
for col in range(original_sheet_COLS):
    worksheet.write(0, col, mag_sheet.row_values(0)[col])
    
# Write each row by iterating column-wise

for row in range(target_sheet_ROWS):
    for col in range(target_sheet_COLS):
        worksheet.write(row+1, col, scienceMagTexts_list[row][col]) # row+1 because of the header
    
    

In [174]:
processed_workbook.close()